# 自定义调用本地大模型方法
1. **类属性定义**:
   - `max_token`: 定义了模型可以处理的最大令牌数。
   - `do_sample`: 指定是否在生成文本时采用采样策略。
   - `temperature`: 控制生成文本的随机性，较高的值会产生更多随机性。
   - `top_p`: 一种替代`temperature`的采样策略，这里设置为0.0，意味着不使用。
   - `tokenizer`: 分词器，用于将文本转换为模型可以理解的令牌。
   - `model`: 存储加载的模型对象。
   - `history`: 存储对话历史。
2. **构造函数**:
   - `__init__`: 构造函数初始化了父类的属性。
3. **属性方法**:
   - `_llm_type`: 返回模型的类型，即`ChatGLM3`。
4. **加载模型的方法**:
   - `load_model`: 此方法用于加载模型和分词器。它首先尝试从指定的路径加载分词器，然后加载模型，并将模型设置为评估模式。这里的模型和分词器是从Hugging Face的`transformers`库中加载的。
5. **调用方法**:
   - `_call`: 一个内部方法，用于调用模型。它被设计为可以被子类覆盖。
   - `invoke`: 这个方法使用模型进行聊天。它接受一个提示和一个历史记录，并返回模型的回复和更新后的历史记录。这里使用了模型的方法`chat`来生成回复，并设置了采样、最大长度和温度等参数。
6. **流式方法**:
   - `stream`: 这个方法允许模型逐步返回回复，而不是一次性返回所有内容。这对于长回复或者需要实时显示回复的场景很有用。它通过模型的方法`stream_chat`实现，并逐块返回回复。

In [1]:
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from langchain_core.messages.ai import AIMessage

class kk_ChatGLM3(LLM):
    max_token: int = 128 * 1024
    do_sample: bool = True
    temperature: float = 0.5
    top_p: float = 0.0
    tokenizer: object = None
    model: object = None
    history: list = []

    def __init__(self):
        super().__init__()

    def load_model(self, model_path: str):
        # 加载分词器
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
        # 加载模型
        model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device_map="mps", use_safetensors=False)
        model = model.eval()
        self.tokenizer = tokenizer
        self.model = model

    def _call(self, prompt: str, config: dict = {}, history: list = []) -> str:
        return self.invoke(prompt, config, history)
    
    def invoke(self, prompt: str, config: dict = {}, history: list = []) -> str:
        # 处理 ChatPromptValue
        if hasattr(prompt, 'to_messages'):
            # 如果是 ChatPromptValue，提取消息内容
            messages = prompt.to_messages()
            prompt = messages[0].content
        elif not isinstance(prompt, str):
            # 尝试转换为字符串
            prompt = str(prompt)

        if not isinstance(history, list):
            history = []
        
        if not isinstance(config, dict):
            config = {}

        response, history = self.model.chat(
            self.tokenizer, 
            prompt, 
            history=history,
            do_sample=self.do_sample,
            temperature=self.temperature,
            max_length=self.max_token,
        )
        self.history = history
        return AIMessage(content=response)

    def stream(self, prompt: str, config: dict = {}, history: list = []) -> str:
        # 处理 ChatPromptValue
        if hasattr(prompt, 'to_messages'):
            # 如果是 ChatPromptValue，提取消息内容
            messages = prompt.to_messages()
            prompt = messages[0].content
        elif not isinstance(prompt, str):
            # 尝试转换为字符串
            prompt = str(prompt)
        
        if not isinstance(history, list):
            history = []
        
        if not isinstance(config, dict):
            config = {}
        preResponese = ""
        for response, _ in self.model.stream_chat(self.tokenizer, prompt):
            if preResponese == "":
                result = response
            else:
                result = response[len(preResponese):]
            preResponese = response
            yield result
    
    @property
    def _llm_type(self) -> str:
        return "kkChatGLM3"
    

/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
llm = kk_ChatGLM3()
model_path = "/Users/libing/kk_LLMs/chatglm3-6b"
llm.load_model(model_path)

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.


Setting unk_token is not supported, use the default one.
/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/libing/miniconda3/envs/kk_agent/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 7/7 [00:02<00:00,  2.63it/s]


In [3]:
# 调用call方法
llm.invoke("中国的首都是哪里？")

AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={})

In [4]:
# 流式输出
for respone in llm.stream("写一首关于75周年国庆节的诗歌"):
    print(respone, end="", flush=True)

今天是国庆节,
75周年庆典,
举国欢腾,
庆祝这个伟大的日子。

75年前,
祖国遭受着入侵,
人民遭受着苦难,
但有一群人,
不畏强敌,
不畏艰险,
用自己的汗水和智慧,
为国家赢得了独立和自由。

75年后,
祖国发生了巨变,
经济繁荣,
文化繁荣,
人民幸福,
国家强大。

今天,
我们欢庆这个伟大的日子,
祝福我们的国家,
祝福我们的人民,
愿我们的国家永远繁荣昌盛,
永远强大。

In [7]:
# 链式输出
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("请根据提示词主题写一篇小红书的营销短文:  {topic}")

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

for chunk in chain.stream({"topic": "讲一个米晓斌投资a股的故事"}):
    print(chunk, end="")

标题：【股市传奇】米晓斌：从新手到A股投资大师，只需一个信念

导语：在繁杂的股海中，每一位投资者都在寻找属于自己的投资之道。而米晓斌，一位普通的中国散户，凭借着自己的信念和坚持，成为了一位传奇的A股投资者。今天，我们就来听听他的投资故事。

正文：

米晓斌，一个普通的上班族，在朋友的推荐下，开始了他的股市之旅。起初，他对股市知之甚少，甚至 considered 自己是一个完全的股市新手。然而，他并没有因此而气馁，反而下定决心，要在这个市场里大显身手。

米晓斌开始通过阅读各种投资书籍和参加投资讲座来提升自己的投资水平。在不断学习和实践的过程中，他逐渐发现，要想在股市中取得成功，关键在于坚持自己的信念，并根据市场情况做出灵活的调整。

有一次，米晓斌看中了一支潜力股，他坚定地相信这只股票的未来前景。尽管朋友们都认为他过于乐观，但他仍然坚定地持有这只股票。最终，这支股票涨了数倍，让米晓斌赚取了可观的利润。

从那以后，米晓斌更加坚定自己的信念，并不断总结自己的投资经验。他认为，只有坚持自己的信念，并根据市场情况做出灵活的调整，才能在股市中取得成功。

最后，米晓斌想送给正在股市中奋斗的朋友们一句名言：“信念是成功的关键。”只要坚定自己的信念，并根据市场情况做出灵活的调整，相信每位投资者都能在股市中取得成功。